# A causal look into the factors of world happiness [2-Causal inference]

## intro
We will put aside causal graphs and Graphical Causal Modeling as experimental practice (see [previous post](https://economyoftime.net/a-causal-look-into-the-factors-of-world-happiness-1-graphical-modeling-5d6ae3a5df1)) for this time and look into Causal Inference itself as presented by Laurence Wong in his [very interesting blogpost](https://laurencewong.com/software/initialization) from 2016. We are going to apply the methods described there to our dataset.

For the sake of this we will have these new mapping:
```
Y: Ladder score
D: "treatment" status indicator, we will try to do some interventions
   on some subset of the records (for example, what happens if income raises...)
   this binary label will be set to 1 for the records that received the intervention.
X: a matrix of all the covariates used in the dataset (Y  S  J  X  W)
```

A score to which has been assigned treatment is `Y(1)` (treated), otherwise non-treated is `Y(0)` (non-treated).

This exercise leverages the `causalinference` Python library.

## load the dataset

In [1]:
import pandas as pd
from causalinference import CausalModel

In [2]:
df = pd.read_csv('world-happiness-report-2021.csv')
df.drop(list(df.filter(regex='Explained')), axis=1, inplace=True)
df.head()

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Dystopia + residual
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,3.253
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.7,0.946,0.030,0.179,2.43,2.868
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.4,0.919,0.025,0.292,2.43,2.839
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.0,0.955,0.160,0.673,2.43,2.967
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.4,0.913,0.175,0.338,2.43,2.798


In [3]:
df = df[["Ladder score",
         "Logged GDP per capita",
         "Social support",
         "Healthy life expectancy",
         "Freedom to make life choices"]
].copy()
df.rename(columns={
    "Ladder score": "Y",
    "Logged GDP per capita": "S",
    "Social support": "J",
    "Healthy life expectancy": "X",
    "Freedom to make life choices": "W" 
}, inplace=True)
df.head(5)

,Y,S,J,X,W
0,7.842,10.775,0.954,72.0,0.949
1,7.620,10.933,0.954,72.7,0.946
2,7.571,11.117,0.942,74.4,0.919
3,7.554,10.878,0.983,73.0,0.955
4,7.464,10.932,0.942,72.4,0.913


In [4]:
df.describe()

,Y,S,J,X,W
count,149.000000,149.000000,149.000000,149.000000,149.000000
mean,5.532839,9.432208,0.814745,64.992799,0.791597
std,1.073924,1.158601,0.114889,6.762043,0.113332
min,2.523000,6.635000,0.463000,48.478000,0.382000
25%,4.852000,8.541000,0.750000,59.802000,0.718000
50%,5.534000,9.569000,0.832000,66.603000,0.804000
75%,6.255000,10.421000,0.905000,69.600000,0.877000
max,7.842000,11.647000,0.983000,76.953000,0.970000


## assumptions
These prerequisites must hold:

1. *randomized experiment* ("strong" prerequisite)

assignment of treatment must be random:
```
(Y(0), Y(1)) ⟂ D
```

2. *unconfounded assumption* ("weak" prerequisite)

exclude confounding among covariates (`X`), there is no unobserved confounder:
```
(Y(0), Y(1)) ⟂ D|X
```
Effects of treatment are orthogonal to treatment conditional covariates.


Spotting confounders is not the subject of this post, see Causal Discovery and Causal Graphs about how to avoid confounding.

## initialisation

Let's assign each record to a "treated group" (`Y(1)`) or to a "control group" (`Y(0)`):

In [5]:
# randomise treatment in the dataset
import numpy as np
df["D"] = np.random.choice(a=[0,1], size=df["Y"].count(), p=[0.4, 0.6])
print(df["D"].to_numpy())

[1 1 1 1 0 0 1 0 0 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1
 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 1 1 1 1 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0
 0 0 1 0 1 0 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 0 0 0
 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 0 1
 0]


In [6]:
## causalinference uses the convention of calling covariates as `Xn` so we rename for convenience
df.rename(columns={
    "S": "X0",
    "J": "X1",
    "X": "X2",
    "W": "X3" 
}, inplace=True)


## intervention 1

Now we simulate the treatment, we increase the "freedom of choice" index (`W` or `X3` depending on which convention we are using) of a given amount only for the treated samples.

First we need to find a way to do that with incurring in errors, let's see how `W` looks like:

In [7]:
df["X3"].describe()

count    149.000000
mean       0.791597
std        0.113332
min        0.382000
25%        0.718000
50%        0.804000
75%        0.877000
max        0.970000
Name: X3, dtype: float64

There is a standard deviation of `0.113332` so we will go for 1/10 of that just to be sure we don't stir the water too much in the beginning. So our treatment looks like:
```
Y(1) => X3 = X3 + (std(X3) / 10)
```

this is the starting scenario:

In [8]:
# let's set aside the initial data and its causal model for comparison
df_start = df.copy()
df_start.head(5)

,Y,X0,X1,X2,X3,D
0,7.842,10.775,0.954,72.0,0.949,1
1,7.620,10.933,0.954,72.7,0.946,1
2,7.571,11.117,0.942,74.4,0.919,1
3,7.554,10.878,0.983,73.0,0.955,1
4,7.464,10.932,0.942,72.4,0.913,0


This is after we apply the treatment ("**intervention 1**": slight increase of freedom of choice):

In [9]:
std_dev_X3 = df_start["X3"].std()
print(std_dev_X3 / 10)

mask = df_start["D"] == 1

df_intervention1 = df_start.copy()
# apply intervention
df_intervention1.loc[mask, 'X3'] = df_intervention1.loc[mask, "X3"].apply(lambda x: x + (std_dev_X3 / 10))
df_intervention1.head()

0.011333178506605257


,Y,X0,X1,X2,X3,D
0,7.842,10.775,0.954,72.0,0.960333,1
1,7.620,10.933,0.954,72.7,0.957333,1
2,7.571,11.117,0.942,74.4,0.930333,1
3,7.554,10.878,0.983,73.0,0.966333,1
4,7.464,10.932,0.942,72.4,0.913000,0


We can see that the samples with intervention (`D=1`) have now the `X3` value increased by a minimal ~0.01.

Let's generate the model for **intervention 1**:

In [10]:
# we simplify the model considering only some covariates
causal_interv1 = CausalModel(
    Y=df_intervention1["Y"].to_numpy(),
    D=df_intervention1["D"].to_numpy(),
    X=df_intervention1[["X0", "X1", "X2", "X3"]].to_numpy()
)


Ready to roll!

Let's see some statistics from the observations.

In [11]:
print(causal_interv1.summary_stats)


Summary Statistics

                        Controls (N_c=55)          Treated (N_t=94)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y        5.634        1.042        5.474        1.093       -0.160

                        Controls (N_c=55)          Treated (N_t=94)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0        9.674        0.980        9.291        1.234       -0.344
             X1        0.842        0.105        0.799        0.118       -0.389
             X2       65.715        6.230       64.570        7.053       -0.172
             X3        0.788        0.121        0.805        0.109        0.152



A lot to go trough here. `N_c` is the size of the control group, `N_t` is the size of the treated group as came out from random sampling. Mean and Standard Deviation are computed for each group for the score and each covariate.

In the upper right `Raw-diff` is the expected difference between treated and non-treated. This is an absolute difference `E[Y(1) - Y(0)]`, so it is not much explicative and also in this case the difference is lower than the standard deviation.

The interesting part is the normalized differences column `Nor-diff` for each covariate. This is the Imbens-Rubin (2015) normalized difference in average covariates. It is a relatively complicated equation that the library compute for us, quite useful, and this is just the beginning. `Nor-diff` is used to spot "covariate imbalance" that happens when treatment and control groups demonstrate insufficient overlap, usually we expect its values to be lower than 0.5 otherwise we may need to apply correction that we are going to explore later like "trimming".

In this case we don't see much of an effect so we will try to "increase the dosage" and push the value to a greater increase.

## intervention 2

We now apply a treatment that is 1/3 of the freedom of choice index:

In [12]:
std_dev_X3 = df_start["X3"].std()
print(std_dev_X3 / 3)

mask = df_start["D"] == 1

df_intervention2 = df_start.copy()
df_intervention2.loc[mask, 'X3'] = df_intervention2.loc[mask, "X3"].apply(lambda x: x + (std_dev_X3 / 3))
df_intervention2.head()

0.03777726168868419


,Y,X0,X1,X2,X3,D
0,7.842,10.775,0.954,72.0,0.986777,1
1,7.620,10.933,0.954,72.7,0.983777,1
2,7.571,11.117,0.942,74.4,0.956777,1
3,7.554,10.878,0.983,73.0,0.992777,1
4,7.464,10.932,0.942,72.4,0.913000,0


Recompute the causal model:

In [13]:
# we simplify the model considering only some covariates
causal_interv2 = CausalModel(
    Y=df_intervention2["Y"].to_numpy(),
    D=df_intervention2["D"].to_numpy(),
    X=df_intervention2[["X0", "X1", "X2", "X3"]].to_numpy()
)

print(causal_interv2.summary_stats)


Summary Statistics

                        Controls (N_c=55)          Treated (N_t=94)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y        5.634        1.042        5.474        1.093       -0.160

                        Controls (N_c=55)          Treated (N_t=94)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0        9.674        0.980        9.291        1.234       -0.344
             X1        0.842        0.105        0.799        0.118       -0.389
             X2       65.715        6.230       64.570        7.053       -0.172
             X3        0.788        0.121        0.832        0.109        0.381



As we can see the index `Nor-diff` for covariate imbalance grew relevantly for `X3` quite close to the 0.5 threshold and that exactly what we were expecting, we pushed `X3` so much that a a light on our dashboard started blinking, probably intervention 2 is too much of a treatment and we risk to invalidate the critical next step that is **Treatment Effect Estimation**.

## Treatment Effect Estimation

In the previous sections we applied two simulated intervention on a covariate, how do we estimate the expected effects of the interventions? `causalinference` gives us some tools, we will look into: "Ordinary Least Square (**OLS**)" and "**Matching**" to estimate the effect of the intervention on the `Ladder Score`.


### OLS
Compute OLS for **intervention 1** and **intervention 2**:

In [14]:
causal_interv1.reset()
causal_interv1.est_via_ols()
print(causal_interv1.estimates)

causal_interv2.reset()
causal_interv2.est_via_ols()
print(causal_interv2.estimates)


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      0.058      0.104      0.553      0.580     -0.147      0.262
           ATC      0.044      0.089      0.492      0.623     -0.131      0.219
           ATT      0.066      0.119      0.552      0.581     -0.168      0.299


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -0.001      0.112     -0.008      0.994     -0.221      0.219
           ATC     -0.028      0.092     -0.299      0.765     -0.209      0.154
           ATT      0.015      0.133      0.111      0.911     -0.246      0.276



/home/lorenzo/code/notepad/.venv/lib/python3.8/site-packages/causalinference/estimators/ols.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  olscoef = np.linalg.lstsq(Z, Y)[0]


OLS applies a linear function with covariates adjustments.

Here we are three foundamental indices that are computed by the estimation:
1. ATE is the Average Treatment Effect: the effect observed in the simulation on the whole dataset
2. ATC is the Average Effect on Controls: the effect observed in the simulation on the control group
3. ATT is the Avergae Effect on Treated: the effect observed in the simulation on treated group

> This result shows that OLS is essentially imputing the missing potential outcomes of a given group by extrapolating linearly from the observations of the other group. It thus follows that the less covariate overlap there is between the two groups the more hopelessly heroic the extrapolation, especially if the underlying relationship between outcomes and covariates is nonlinear to begin with. (Laurence Wong)

There are plenty of considerations to keep in mind when reading at these numbers. Looking at the column `Est.` you can have a very rough idea of what is going on between the covariate object of the intervention and the effect. Something we can do is to try a different effect estimation algorithm to see if the extrapolation run with OLS can be adjusted somehow.

### Matching
We can try to compute the estimation on matching records with similar covariate values. We apply the linear extrapolation only on records that have similar characteristics:

In [15]:
causal_interv1.reset()
causal_interv1.est_via_matching(bias_adj=True)
print(causal_interv1.estimates)


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -0.118      0.151     -0.785      0.433     -0.414      0.177
           ATC     -0.020      0.167     -0.119      0.905     -0.348      0.308
           ATT     -0.176      0.176     -0.997      0.319     -0.521      0.170



/home/lorenzo/code/notepad/.venv/lib/python3.8/site-packages/causalinference/estimators/matching.py:100: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return np.linalg.lstsq(X, Y)[0][1:]  # don't need intercept coef


In [16]:
causal_interv2.reset()
causal_interv2.est_via_matching(bias_adj=True)
print(causal_interv2.estimates)


Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -0.100      0.138     -0.729      0.466     -0.370      0.170
           ATC     -0.049      0.170     -0.286      0.775     -0.383      0.285
           ATT     -0.131      0.153     -0.854      0.393     -0.430      0.169






We can easily say one thing though, in this estimation we took the whole of the dataset without considering the substantial differences among the records. We may have ended up adding a bias to the estimation by considering all the records at the same time. We are going to apply "stratification" so we can see if the effect estimation changes when applied to more homogeneous subgroups in the dataset.

## propensity score

We are going to create strata in the dataset so to mitigate possible biases. Before we need to introduve the concept of **propensity score**.

> The probability of receiving treatment, also known as the propensity score, plays a very special role in the estimation of treatment effects. (Laurence Wong)

> The propensity score is the conditional probability of receiving the treatment given the observed covariates. Estimation is done via a logistic regression. (`est_propensity` docstring)

Thanks to the unconfounded assumptions (see section above) and the work done by Rosenbaum-Rubin (1983), we  can safely assume that this reasoning follows:
```
# unconfoundedness assumption
(Y(0), Y(1)) ⟂ D
# implies according to Rosembaum-Rubin 
(Y(0), Y(1)) ⟂ D | p(X)
# that is
p(X) = P(D=1|X)
```
This allows to define an algorithm Imbens-Rubin(2015) for variable selection for estimating the propensity score. The propensity score is foundamental to refine the estimation techniques for causal effects.

Let's try to compute the propensity score for the original dataset:

In [17]:
causal_start = CausalModel(
    Y=df_start["Y"].to_numpy(),
    D=df_start["D"].to_numpy(),
    X=df_start[["X0", "X1", "X2", "X3"]].to_numpy()
)

causal_start.est_via_ols()
causal_start.est_via_matching(bias_adj=True)
print(causal_start.estimates)


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      0.083      0.101      0.818      0.413     -0.116      0.281
           ATC      0.075      0.089      0.841      0.401     -0.099      0.249
           ATT      0.088      0.114      0.771      0.441     -0.135      0.311

Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE     -0.056      0.153     -0.363      0.717     -0.356      0.245
           ATC      0.019      0.178      0.105      0.916     -0.330      0.367
           ATT     -0.099      0.174     -0.571      0.568     -0.440      0.241



/home/lorenzo/code/notepad/.venv/lib/python3.8/site-packages/causalinference/estimators/ols.py:21: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  olscoef = np.linalg.lstsq(Z, Y)[0]
/home/lorenzo/code/notepad/.venv/lib/python3.8/site-packages/causalinference/estimators/matching.py:100: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  return np.linalg.lstsq(X, Y)[0][1:]  # don't need intercept coef


In [18]:
print("Appliying only linear logistic regressions")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
causal_start.est_propensity(lin="all")
print(causal_start.propensity)

print("Appliying linear logistic regressions and quadratic for X3 and X3*X1")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
causal_start.est_propensity(lin="all", qua=[(3,3), (1,3)])
print(causal_start.propensity)

Appliying only linear logistic regressions
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      1.724      1.791      0.962      0.336     -1.787      5.234
            X0     -0.428      0.342     -1.254      0.210     -1.098      0.241
            X1     -4.806      2.812     -1.709      0.087    -10.318      0.706
            X2      0.071      0.052      1.366      0.172     -0.031      0.174
            X3      2.747      1.832      1.500      0.134     -0.843      6.337

Appliying linear logistic regressions and quadratic for X3 and X3*X1
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
-------------------------------------------------------------------

The search for a good combination of linear (`XN`) and quadratic (`XN*XN`) terms for the logistic regression to compute the propensity score is a foundamental way for understanding how the covariates influence each others [1][2].

Remember that (see [link](https://stats.stackexchange.com/q/412668)):
> Negative coefficients in a logistic regression model translate into odds ratios that are less than one (viz., (0,1)). That in turn, means that the predicted probability is decreasing as the covariate increases.

Let's see how this number changes after intervention 2:

In [19]:
print("Appliying only linear logistic regressions")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
causal_interv1.est_propensity(lin="all")
print(causal_interv1.propensity)


Appliying only linear logistic regressions
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      1.404      1.797      0.781      0.435     -2.118      4.927
            X0     -0.427      0.345     -1.237      0.216     -1.103      0.249
            X1     -5.280      2.845     -1.856      0.063    -10.856      0.295
            X2      0.067      0.053      1.276      0.202     -0.036      0.171
            X3      3.947      1.848      2.136      0.033      0.325      7.569



In [20]:

print("Appliying linear logistic regressions and quadratic for X3 and X3*X1")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
causal_interv2.est_propensity(lin="all", qua=[(3,3), (1,3)])
print(causal_interv2.propensity)


Appliying linear logistic regressions and quadratic for X3 and X3*X1
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept     -0.397      7.390     -0.054      0.957    -14.881     14.087
            X0     -0.397      0.360     -1.100      0.271     -1.103      0.310
            X1      1.554     13.822      0.112      0.911    -25.538     28.645
            X2      0.053      0.054      0.968      0.333     -0.054      0.159
            X3      1.106     14.977      0.074      0.941    -28.248     30.460
         X3*X3      9.124     14.785      0.617      0.537    -19.854     38.103
         X1*X3    -10.425     18.064     -0.577      0.564    -45.831     24.981



In [21]:


print("Appliying Imbens-Rubin(2015) algorithm")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
causal_interv2.est_propensity_s()
print(causal_interv2.propensity)

Appliying Imbens-Rubin(2015) algorithm
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      0.998      1.567      0.637      0.524     -2.073      4.069
            X1     -7.276      2.096     -3.471      0.001    -11.385     -3.167
            X3      6.807      1.908      3.569      0.000      3.068     10.546



Note down the differences you notice from before and after intervention 2 and which covariates get influenced by the intervention in `X3`.

### Trimming
Set a cut-off value to drop values with the propensity score outside a given interval:

In [22]:
# default cut-off is 0.1: only PS between .1 and .9 are considered 
# because the min PS is 0.26, we need to set the cut-off high enough
causal_start.cutoff = 0.30

# before trimming
causal_start.reset()
causal_start.est_propensity_s()
causal_start.trim()
print(causal_start.propensity)


Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      2.435      1.522      1.600      0.110     -0.548      5.418
            X1     -5.159      1.947     -2.649      0.008     -8.976     -1.342
            X3      2.960      1.796      1.648      0.099     -0.560      6.480



The dataset looks to be sensitive to dropping tail values.

## Stratification
Another usefull usage of the propensity score is to create buckets of records with inside some interval of PS. With stratification each block is a group in a propensity score interval:

In [23]:
causal_start.reset()
causal_start.est_propensity_s()

causal_start.stratify_s()
print(causal_start.strata)


Stratification Summary

              Propensity Score         Sample Size     Ave. Propensity   Outcome
   Stratum      Min.      Max.  Controls   Treated  Controls   Treated  Raw-diff
--------------------------------------------------------------------------------
         1     0.405     0.624        34        41     0.537     0.548     0.228
         2     0.629     0.904        21        53     0.696     0.729    -0.235



After stratification we are able to compute cleaner estimates for each stratum. This leads us to Blocking.

## Blocking
Aggregating strata estimates of treatment effects gives the *blovking estimator*, computed by the ATE of every stratum.

With OLS:

In [24]:
causal_start.reset()
causal_start.est_propensity_s()
causal_start.est_via_ols()
print(causal_start.propensity)
print(causal_start.estimates)



Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      2.435      1.522      1.600      0.110     -0.548      5.418
            X1     -5.159      1.947     -2.649      0.008     -8.976     -1.342
            X3      2.960      1.796      1.648      0.099     -0.560      6.480


Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      0.083      0.101      0.818      0.413     -0.116      0.281
           ATC      0.075      0.089      0.841      0.401     -0.099      0.249
           ATT      0.088      0.114      0.771      0.441     -0.135      0.311



With Blocking:

In [25]:

causal_start.reset()
causal_start.est_propensity_s()
causal_start.stratify()  # without trimming
causal_start.est_via_blocking()
print(causal_start.propensity)
print(causal_start.estimates)


Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept      2.435      1.522      1.600      0.110     -0.548      5.418
            X1     -5.159      1.947     -2.649      0.008     -8.976     -1.342
            X3      2.960      1.796      1.648      0.099     -0.560      6.480


Treatment Effect Estimates: Blocking

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      0.007      0.101      0.073      0.942     -0.190      0.205
           ATC     -0.021      0.079     -0.269      0.788     -0.177      0.134
           ATT      0.024      0.118      0.204      0.838     -0.208      0.256



We can se the debiasing effect performed by stratification and better estimates.

## Conclusions

With these formally demonstrated tools we should be able to have a clearer understanding about the relations in place with the covariates, permutating over these exercise testing different assumptions can result into new debiased insights. With these new clues we can proceed more confident to a summary of our analysis or rather try to update our causal graph considering the influence that every variable can have on each other, maybe removing edges or establishing colliders and y-shapes in our graph. These methods for relations discovery can be automated via algorithms, that is what we are going to try in the next post.

As an exercise you can try to answer why the causal graph below is, compared to the ones proposed in the previous posts, a better representations of the dataset according to what we have seen until here:

![causal graph](./assets/CausalGraph-post-2.png)

In particular can we weaken the probability of possible causal relation between `W` and `Y` according to the dataset?


## Reference
[1] Kuss, Oliver. “The z-difference can be used to measure covariate balance in matched propensity score analyses.” Journal of clinical epidemiology vol. 66,11 (2013): 1302-7. doi:10.1016/j.jclinepi.2013.06.001

[2] Austin PC. An Introduction to Propensity Score Methods for Reducing the Effects of Confounding in Observational Studies. Multivariate Behav Res. 2011;46(3):399-424. doi:10.1080/00273171.2011.568786

There are plenty of papers and dataset to tackle Causal Inference, you can start from [this list](https://github.com/matthewvowels1/Awesome-Causal-Inference)